In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.3.5'

In [53]:
G_path = '/Users/sepehrbe/My_Drive/DataSources/SkinCare'
import numpy as np
import os
import torch
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from statistics import mean
import sklearn as sk
import warnings
from sklearn.utils import resample
import pickle as pickle
import PIL as pl
from PIL.Image import Transpose
from PIL import Image
from usefull_functions import *
warnings.filterwarnings('ignore')

# **Import Meta Data**

In [12]:
file = 'HAM10000_metadata.csv'
meta_data_v0 = pd.read_csv(G_path + '/archive/' + file)

In [13]:
# Remove Duplication
meta_data_noDup = meta_data_v0.groupby('lesion_id').first().reset_index()
# Encode Labels
meta_data_noDup.dx = pd.Categorical(meta_data_noDup.dx)
meta_data_noDup['image_label'] = meta_data_noDup.dx.cat.codes

remap = {0:1 , 1:1, 4:1, 2:0, 3:0, 5:0, 6:0}
meta_data_noDup['binary_label'] = meta_data_noDup['image_label']
meta_data_noDup = meta_data_noDup.replace({'binary_label':remap})

meta_data_noDup.to_csv(G_path + 'Meta_data_noDup.csv')
### insert the code of R for Age filling

######################
filled_nan_ages = pd.read_csv(G_path+'/synthetics_ages.csv').rename(columns={'age':'age_mdf'})

meta_data_noDup = meta_data_noDup.merge(filled_nan_ages,how='left', on='lesion_id')
meta_data_noDup['age'] = meta_data_noDup.apply(lambda x: x.age if pd.notnull(x.age) else x.age_mdf, axis=1)

try: meta_data_noDup.drop(columns=['Unnamed: 0','age_mdf'],inplace=True)
except: None
# Reduce Nr. class 5 category to 2800
class_5 = meta_data_noDup[meta_data_noDup.image_label==5]
class_5_reduced = class_5.sample(n=2000, random_state=0)
meta_data_noDup_reduced = pd.concat([meta_data_noDup[meta_data_noDup.image_label!=5],class_5_reduced],axis=0)
# Get 
train_meta, test_valid_meta = train_test_split(meta_data_noDup_reduced, test_size = 0.2, random_state=0,stratify=meta_data_noDup_reduced['image_label'])
test_meta, valid_meta = train_test_split(test_valid_meta, test_size = 0.5, random_state=0,stratify=test_valid_meta['image_label'])

In [56]:
meta_data_noDup.image_label.value_counts(normalize=True)

5    0.723293
2    0.097323
4    0.082195
1    0.043775
0    0.030522
6    0.013119
3    0.009772
Name: image_label, dtype: float64

In [58]:
Meta_Data_Augmented.image_label.value_counts(normalize=True)

5    0.430385
2    0.156445
4    0.132128
1    0.126748
0    0.088229
6    0.037874
3    0.028190
Name: image_label, dtype: float64

In [14]:
train_meta['type'] = 'train'
valid_meta['type'] = 'valid'
test_meta['type'] = 'test'

In [43]:
blc_test_meta = pd.concat([valid_meta,test_meta],axis=0)
# blc_test_meta = blc_test_meta.groupby('image_label').head(15).reset_index()
blc_test_meta = blc_test_meta.groupby('image_label').sample(n=15,random_state=0)
pickle.dump(blc_test_meta , open(G_path + '/blc_test_meta_list','wb'))

# **Import Images**

In [16]:
img_list = []
for path, dirs, files in  os.walk(G_path + '/archive'):
    for f in files:
        img_list.append( path +'/' + f)
    for d in dirs:
        img_list.append( path + d)
img_list = list(set([x for x in img_list if ".jpg" in x] ))

print(len(img_list))

10015


In [17]:
# Copy Images for Augmentation Directory
Augment_Path = G_path + '/' + 'Augmentation'
try:
    os.makedirs(Augment_Path)
except:
    None
try:
    os.makedirs(Augment_Path+'/test')
    os.makedirs(Augment_Path+'/valid')
    os.makedirs(Augment_Path+'/train')
except:
    None
labels = list(train_meta.image_label.unique())

for i in labels:
    try:
        os.makedirs(Augment_Path+'/test/'+str(i))
        os.makedirs(Augment_Path+'/valid/'+str(i))
        os.makedirs(Augment_Path+'/train/'+str(i))
    except:
        None

In [18]:
data_ = train_meta[['image_id','image_label']]
name = 'train'
def copy_to_AutoEncoder(data_ , name):
    id_list = list(data_['image_id'])
    id_labels = list(data_['image_label'])
    for i in range(len(id_list)):
        for j in img_list:
            if id_list[i] in j:
                shutil.copy(j, Augment_Path+'/' + name+ '/' + str(id_labels[i]) + '/' + id_list[i] + '.jpg')
                break

data_ = train_meta[['image_id','image_label']]
name = 'train'
copy_to_AutoEncoder(data_ = data_ , name = name)
data_ = valid_meta[['image_id','image_label']]
name = 'valid'
copy_to_AutoEncoder(data_ = data_ , name = name)
data_ = test_meta[['image_id','image_label']]
name = 'test'
copy_to_AutoEncoder(data_ = data_ , name = name)

# **Data Augmentation**

In [19]:
folder_dir = Augment_Path + '/train/'

for i in [3, 6, 0, 1]:
    f = folder_dir + str(i) + "/"
    for image in os.listdir(f):
        if "ISIC" in image and "AUG_" not in image and image.replace(".jpg", ""):
            # print("Augmented image: ", image)
            im1 = Image.open(f + image)
            augmented = im1.transpose(Transpose.FLIP_LEFT_RIGHT)
            augmented = augmented.transpose(Transpose.FLIP_TOP_BOTTOM)
            augmented.save(folder_dir + "/" + str(i) + "/AUG_" + image)
            # Add metadata of Augmented image
            new_row = train_meta[train_meta.image_id == image.replace(".jpg", "")]
            new_row['image_id'] = 'AUG_' + new_row['image_id'].iloc[0]
            train_meta = train_meta.append(new_row)

In [23]:
Meta_Data_Augmented = pd.concat([train_meta, valid_meta, test_meta], axis=0)
pickle.dump(Meta_Data_Augmented, open( G_path + '/Meta_Data_Augmented','wb'))

# **Import Augmented Images and rescaling to 280x210**

In [24]:
img_list = []
for path, dirs, files in  os.walk(Augment_Path):
    for f in files:
        img_list.append( path +'/' + f)
    for d in dirs:
        img_list.append( path + d)
img_list = list(set([x for x in img_list if ".jpg" in x] ))

Images_280x210 = G_path + '/Images_280x210'

try:
    os.makedirs(Images_280x210)
except:
    None

for img in tqdm(img_list):
    file_out = Images_280x210 +'/'+ re.findall('\w+',img)[-2:-1][0] +'.jpg'
    os.system('convert %s %s %s' % ('-resize 46.66%',img,file_out))

100%|██████████| 4647/4647 [01:31<00:00, 50.99it/s]


# **Import Rescaled Images and convert to arrays**

In [25]:
img_list = os.listdir(Images_280x210)
Augmented_280x210_dataset = pd.DataFrame(columns=['image_id','img_array'])
for img in tqdm(img_list):
    if '.jpg' in img:
        img_name = img.replace('.jpg','')
        image = pl.Image.open( Images_280x210 +'/'+ img)
        img_array = np.asarray(image)
        new_row = pd.Series({'image_id':img_name,'img_array':img_array}, name='')
        Augmented_280x210_dataset = Augmented_280x210_dataset.append(new_row)

100%|██████████| 4648/4648 [00:09<00:00, 480.26it/s]


In [26]:
CNN_280x210_dataset = Augmented_280x210_dataset.merge(Meta_Data_Augmented, how="left",on='image_id')
blc_test_dataset_280x210 = Augmented_280x210_dataset[['image_id','img_array']].merge(blc_test_meta, how="right",on='image_id')

# **CNN_280x210: Standardization / Normalization**

In [27]:
df_ = CNN_280x210_dataset
train_data = df_[df_.type=='train']
valid_data = df_[df_.type=='valid']
test_data = df_[df_.type=='test']
print('Train size: ',train_data.shape[0] ,' Valid Size: ',valid_data.shape[0], ' Test size: ', test_data.shape[0])

Train size:  3833  Valid Size:  407  Test size:  407


In [28]:
training_data = pd.concat([train_data,valid_data], axis=0)
training_arrays_scaled = np.stack(training_data['img_array'].values)  / 255
Mean = training_arrays_scaled.mean(axis = (0,1,2)) 
STD = training_arrays_scaled.std(axis = (0,1,2))
print(f"Mean : {Mean}   STD: {STD}") 

Mean : [0.76570269 0.55224556 0.57657015]   STD: [0.13616887 0.1481389  0.16343116]


In [30]:
#Train data standardization
train_arrays_std = np.stack(train_data['img_array'].values) / 255
for i in range(0,train_arrays_std.shape[0]):
    train_arrays_std[i] = (train_arrays_std[i] - Mean) / STD
#Validation data standardization
valid_arrays_std = np.stack(valid_data['img_array'].values) / 255
for i in range(0,valid_arrays_std.shape[0]):
    valid_arrays_std[i] = (valid_arrays_std[i] - Mean) / STD
#Test data standardization
test_arrays_std = np.stack(test_data['img_array'].values) / 255
for i in range(0,test_arrays_std.shape[0]):
    test_arrays_std[i] = (test_arrays_std[i] - Mean) / STD

#Test-Valid data standardization
test_valid_arrays_std = np.stack(blc_test_dataset_280x210['img_array'].values) / 255
for i in range(0,test_valid_arrays_std.shape[0]):
    test_valid_arrays_std[i] = (test_valid_arrays_std[i] - Mean) / STD

In [31]:
Mean_t = test_arrays_std.mean(axis = (0,1,2)) 
STD_t = test_arrays_std.std(axis = (0,1,2))
print(f"Test Set Mean : {Mean_t}   Test Set STD: {STD_t}") 

Test Set Mean : [-0.01839491 -0.04231122 -0.04687724]   Test Set STD: [0.96394774 0.99849143 1.00558878]


In [32]:
metda_keys = train_data[['type','image_id']].reset_index(drop=True), valid_data[['type','image_id']].reset_index(drop=True), test_data[['type','image_id']].reset_index(drop=True)

train_arrays_std_T = torch.tensor(train_arrays_std, dtype= torch.float32).transpose(3,1)
train_labels = torch.tensor(train_data['image_label'].values.astype(np.long))

valid_arrays_std_T = torch.tensor(valid_arrays_std, dtype= torch.float32).transpose(3,1)
valid_labels = torch.tensor(valid_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_valid_arrays_std_T = torch.tensor(test_valid_arrays_std, dtype= torch.float32).transpose(3,1)
test_valid_labels = torch.tensor(blc_test_dataset_280x210['image_label'].values.astype(np.long))

train_labels_binary =  torch.tensor(label_to_binary(train_labels))
valid_labels_binary =  torch.tensor(label_to_binary(valid_labels))
test_labels_binary =  torch.tensor(label_to_binary(test_labels))
test_valid_labels_binary =  torch.tensor(label_to_binary(test_valid_labels))


input_data = (train_arrays_std_T , valid_arrays_std_T, test_arrays_std_T ,metda_keys)
labels = ((train_labels,valid_labels,test_labels), (train_labels_binary , valid_labels_binary ,test_labels_binary))

balc_evaluation_set = (test_valid_arrays_std_T,test_valid_labels,test_valid_labels_binary)

In [33]:
dataset_training = G_path + '/DataSet_280x210'

try:
    os.makedirs(dataset_training)
except:
    None

pickle.dump((input_data,labels), open(dataset_training + '/DataSet_280x210', 'wb'))
pickle.dump(balc_evaluation_set, open(dataset_training + '/Evaluation_set_280x210', 'wb'))

# **Import AutoEncoders Images**

In [44]:
def import_imges(img_path):
    img_list = []
    for path, dirs, files in  os.walk(img_path):
        for f in files:
            img_list.append( path +'/' + f)
        for d in dirs:
            img_list.append( path + d)
    img_list = list(set([x for x in img_list if ".jpg" in x] ))

    dataset = pd.DataFrame(columns=['image_id','img_array'])
    for img in tqdm(img_list):
        img_name = re.findall('\w+', img)[-2:-1][0]
        image = pl.Image.open( img)
        img_array = np.asarray(image)
        new_row = pd.Series({'image_id':img_name,'img_array':img_array}, name='')
        dataset = dataset.append(new_row)
    return dataset

train_path = G_path + '/Augmented_AutoEncoded/train'
valid_path = G_path + '/Augmented_AutoEncoded/valid'
test_path = G_path + '/Augmented_AutoEncoded/test'

train_set = import_imges(train_path)
valid_set = import_imges(valid_path)
test_set = import_imges(test_path)

100%|██████████| 407/407 [00:00<00:00, 762.57it/s]


In [45]:
dataset_ = pd.concat([train_set,valid_set,test_set],axis=0)
A_E_dataset = dataset_.merge(Meta_Data_Augmented, how='left', on='image_id')
blc_test_dataset_A_E = dataset_[['image_id','img_array']].merge(blc_test_meta, how="right",on='image_id')

In [46]:
df_ = A_E_dataset
train_data = df_[df_.type=='train']
valid_data = df_[df_.type=='valid']
test_data = df_[df_.type=='test']
print('Train size: ',train_data.shape[0] ,' Valid Size: ',valid_data.shape[0], ' Test size: ', test_data.shape[0])

Train size:  3833  Valid Size:  407  Test size:  407


In [47]:
training_data = pd.concat([train_data,valid_data], axis=0)
training_arrays_scaled = np.stack(training_data['img_array'].values)  / 255
Mean = training_arrays_scaled.mean(axis = (0,1,2)) 
STD = training_arrays_scaled.std(axis = (0,1,2))
print(f"Mean : {Mean}   STD: {STD}") 

Mean : [0.36115692 0.34408239 0.32405245]   STD: [0.16846211 0.26209011 0.24330585]


In [48]:
#Train data standardization
train_arrays_std = np.stack(train_data['img_array'].values) / 255
for i in range(0,train_arrays_std.shape[0]):
    train_arrays_std[i] = (train_arrays_std[i] - Mean) / STD
#Validation data standardization
valid_arrays_std = np.stack(valid_data['img_array'].values) / 255
for i in range(0,valid_arrays_std.shape[0]):
    valid_arrays_std[i] = (valid_arrays_std[i] - Mean) / STD
#Test data standardization
test_arrays_std = np.stack(test_data['img_array'].values) / 255
for i in range(0,test_arrays_std.shape[0]):
    test_arrays_std[i] = (test_arrays_std[i] - Mean) / STD

#Test-Valid data standardization
test_valid_arrays_std = np.stack(blc_test_dataset_A_E['img_array'].values) / 255
for i in range(0,test_valid_arrays_std.shape[0]):
    test_valid_arrays_std[i] = (test_valid_arrays_std[i] - Mean) / STD

In [49]:
Mean_t = test_arrays_std.mean(axis = (0,1,2)) 
STD_t = test_arrays_std.std(axis = (0,1,2))
print(f"Test Set Mean : {Mean_t}   Test Set STD: {STD_t}") 

Test Set Mean : [-0.04389305 -0.05144053  0.03501551]   Test Set STD: [0.93557654 0.98728975 1.00074642]


In [50]:
metda_keys = train_data[['type','image_id']].reset_index(drop=True), valid_data[['type','image_id']].reset_index(drop=True), test_data[['type','image_id']].reset_index(drop=True)

In [51]:
train_arrays_std_T = torch.tensor(train_arrays_std, dtype= torch.float32).transpose(3,1)
train_labels = torch.tensor(train_data['image_label'].values.astype(np.long))

valid_arrays_std_T = torch.tensor(valid_arrays_std, dtype= torch.float32).transpose(3,1)
valid_labels = torch.tensor(valid_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_valid_arrays_std_T = torch.tensor(test_valid_arrays_std, dtype= torch.float32).transpose(3,1)
test_valid_labels = torch.tensor(blc_test_dataset_A_E['image_label'].values.astype(np.long))

A_E_input_data = (train_arrays_std_T , valid_arrays_std_T, test_arrays_std_T ,metda_keys)

train_labels_binary =  torch.tensor(label_to_binary(train_labels))
valid_labels_binary =  torch.tensor(label_to_binary(valid_labels))
test_labels_binary =  torch.tensor(label_to_binary(test_labels))

A_E_labels = ((train_labels,valid_labels,test_labels), (train_labels_binary , valid_labels_binary ,test_labels_binary))

A_E_balc_test_set = (test_valid_arrays_std_T,test_valid_labels)

In [52]:
pickle.dump((A_E_input_data,A_E_labels), open(G_path + '/Augmented_AutoEncoded/'+ 'Latent_Input_Data_labels', 'wb'))
pickle.dump(A_E_balc_test_set, open(G_path +  '/Augmented_AutoEncoded/'+  '/Evaluation_set_AutoEncoder', 'wb'))